In [1]:
from pulp import *
import pandas as pd

In [4]:

I = ["NY_reg", "NY_half", "Rm", "Mh", "Ba", "Va"]  # set of facilities
J = ["Missi", "Bramt"]  # set of DCs
F = {
    "NY_reg": 315,    # Regular fixed cost for 'NY'
    "NY_half": 157.5,  # Half capacity fixed cost for 'NY'
    "Rm": 305,
    "Mh": 295,
    "Ba": 160,
    "Va": 200
}#Fixed cost of facilities
C = {
    "NY_reg": 12.5,   # Regular capacity for 'NY'
    "NY_half": 6.25,  # Half capacity for 'NY'
    "Rm": 15,
    "Mh": 21,
    "Ba": 27.5,
    "Va": 25
}
SC = {
    'NY_reg': {'Missi': 5, 'Bramt': 6},    # Shipping costs from 'NY_reg'
    'NY_half': {'Missi': 5, 'Bramt': 6},  # Shipping costs from 'NY_half'
    'Rm': {'Missi': 7, 'Bramt': 8},
    'Mh': {'Missi': 12, 'Bramt': 9},
    'Ba': {'Missi': 17, 'Bramt': 15},
    'Va': {'Missi': 4, 'Bramt': 3}
}
D={'Missi':29.5, 'Bramt':12.5} # demands of the DCs

In [8]:
F

{'NY_reg': 315, 'NY_half': 157.5, 'Rm': 305, 'Mh': 295, 'Ba': 160, 'Va': 200}

In [9]:
df_SC = pd.DataFrame(SC)
df_SC

,NY_reg,NY_half,Rm,Mh,Ba,Va
Missi,5,5,7,12,17,4
Bramt,6,6,8,9,15,3


In [ ]:
model = LpProblem("Production_Facilities_Multiple", LpMinimize)

In [ ]:
x = LpVariable.dicts("x", (I, J), lowBound=0, cat='Continuous')
y = LpVariable.dicts("y", I, 0, 1, LpBinary)

In [ ]:
model += lpSum(F[i] * y[i] for i in I) + lpSum(SC[i][j] * x[i][j] for i in I for j in J), "Obj"

In [ ]:
model += y["NY_reg"] + y["NY_half"] <= 1, "Only one option should be selected"

In [ ]:
for i in I:
    model += lpSum(x[i][j] for j in J) <= C[i] * y[i], f"Satisfy_Capacity_{i}"

In [ ]:
for j in J:
    model += lpSum(x[i][j] for i in I) == 1

In [ ]:
for j in J:
    for i in I:
        model += x[i][j] <= y[i], f"m-Must be open to supply {i}{j}"

In [ ]:
for j in J:
    model += lpSum(x[i][j] for i in I) >= D[j], f"Satisfy demands of {j}"

In [ ]:
model.writeLP("Production_Facilities_Multiple")

[x_Ba_Bramt,
 x_Ba_Missi,
 x_Mh_Bramt,
 x_Mh_Missi,
 x_NY_half_Bramt,
 x_NY_half_Missi,
 x_NY_reg_Bramt,
 x_NY_reg_Missi,
 x_Rm_Bramt,
 x_Rm_Missi,
 x_Va_Bramt,
 x_Va_Missi,
 y_Ba,
 y_Mh,
 y_NY_half,
 y_NY_reg,
 y_Rm,
 y_Va]

In [ ]:
model.solve()

-1

In [ ]:
print("Status:", model.status)
print("Optimal Solution:")
# Print values of x
for i in I:
    for j in J:
        print(f"x[{i}][{j}] =", x[i][j].varValue)
# Print values of y
for i in I:
    print(f"y[{i}] =", y[i].varValue)
    
print("Optimal Profit =", model.objective.value())

Status: -1
Optimal Solution:
x[NY_reg][Missi] = 29.5
x[NY_reg][Bramt] = 0.0
x[NY_half][Missi] = 0.0
x[NY_half][Bramt] = 0.0
x[Rm][Missi] = 0.0
x[Rm][Bramt] = 0.0
x[Mh][Missi] = 0.0
x[Mh][Bramt] = 0.0
x[Ba][Missi] = 0.0
x[Ba][Bramt] = 0.0
x[Va][Missi] = 0.0
x[Va][Bramt] = 0.0
y[NY_reg] = 0.0
y[NY_half] = 0.0
y[Rm] = 0.0
y[Mh] = 0.0
y[Ba] = 0.0
y[Va] = 0.0
Optimal Profit = 147.5
